# Content-based Filtering Soptify Song Recommendation System


In [7]:
# Import library
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
playlistDF = pd.read_csv("../data/processed_data.csv")
print(playlistDF.columns)
playlistDF.drop(columns=["Unnamed: 0",'Unnamed: 0.1'], inplace = True)
playlistDF

Index(['Unnamed: 0', 'Unnamed: 0.1', 'pos', 'artist_name', 'track_uri',
       'artist_uri', 'track_name', 'album_uri', 'duration_ms_x', 'album_name',
       'name', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms_y', 'time_signature', 'artist_pop', 'genres', 'track_pop'],
      dtype='object')


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,danceability,...,type,id,uri,track_href,analysis_url,duration_ms_y,time_signature,artist_pop,genres,track_pop
0,0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,0.904,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,dance pop,69
1,73,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,w o r k o u t,0.904,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,dance pop,69
2,14,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,party playlist,0.904,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,dance pop,69
3,42,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Dance mix,0.904,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,dance pop,69
4,1,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,spin,0.904,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,dance pop,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,11,Zion.T,6yEfq48XkEzCg0TwY6Hhkb,spotify:artist:5HenzRvMtSrgtvU16XAoby,No Make Up,spotify:album:5vudrVEnoTqgAgATgafUL2,189154,No Make Up - Single,korean,0.758,...,audio_features,6yEfq48XkEzCg0TwY6Hhkb,spotify:track:6yEfq48XkEzCg0TwY6Hhkb,https://api.spotify.com/v1/tracks/6yEfq48XkEzC...,https://api.spotify.com/v1/audio-analysis/6yEf...,189154,4,62,k-pop,0
786,12,KARD,1ZgfAxHQCXLt8o1VXEHHAt,spotify:artist:2JhAlkmukNvarUpGhTFXUQ,Oh NaNa (Hidden. HUR YOUNG JI),spotify:album:2HpryOOYLbnST4aCT5v4lF,206772,"K.A.R.D Project Vol.1 ""Oh NaNa""",korean,0.756,...,audio_features,1ZgfAxHQCXLt8o1VXEHHAt,spotify:track:1ZgfAxHQCXLt8o1VXEHHAt,https://api.spotify.com/v1/tracks/1ZgfAxHQCXLt...,https://api.spotify.com/v1/audio-analysis/1Zgf...,206773,4,59,k-pop,56
787,13,KARD,4b1sQpvL7QVgamRZ74F1oA,spotify:artist:2JhAlkmukNvarUpGhTFXUQ,Don't Recall,spotify:album:4wTkdR3v9qFdRuCf2hpBZn,208666,"K.A.R.D Project Vol.2 ""Don't Recall""",korean,0.694,...,audio_features,4b1sQpvL7QVgamRZ74F1oA,spotify:track:4b1sQpvL7QVgamRZ74F1oA,https://api.spotify.com/v1/tracks/4b1sQpvL7QVg...,https://api.spotify.com/v1/audio-analysis/4b1s...,208667,4,59,k-pop,56
788,14,Zion.T,0AO2yQNGcH0ASHHU5lS4lT,spotify:artist:5HenzRvMtSrgtvU16XAoby,양화대교 (Yanghwa Brdg),spotify:album:06ngphQtoWwkns4H1tpXnq,229349,양화대교 (Yanghwa Brdg) - Single,korean,0.538,...,audio_features,0AO2yQNGcH0ASHHU5lS4lT,spotify:track:0AO2yQNGcH0ASHHU5lS4lT,https://api.spotify.com/v1/tracks/0AO2yQNGcH0A...,https://api.spotify.com/v1/audio-analysis/0AO2...,229350,4,62,k-pop,0


In [26]:
playlistDF[['artist_name','track_name','name']]

,artist_name,track_name,name
0,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),Throwbacks
1,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),w o r k o u t
2,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),party playlist
3,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),Dance mix
4,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),spin
...,...,...,...
785,Zion.T,No Make Up,korean
786,KARD,Oh NaNa (Hidden. HUR YOUNG JI),korean
787,KARD,Don't Recall,korean
788,Zion.T,양화대교 (Yanghwa Brdg),korean


In [23]:
playlistDF['artists_song'] = playlistDF.apply(lambda row: row['artist_name']+row['track_name'],axis = 1)
songDF = playlistDF.drop_duplicates('artists_song')
songDF[['artist_name','id','track_name','duration_ms_x','duration_ms_y','album_name','danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','time_signature', "artist_pop", "genres", "track_pop"]]
songDF.to_csv("../data/allsong_data.csv", index = False)

In [27]:
songDF.genres[0]


'dance pop'

In [19]:
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(songDF['genres'].apply(lambda x: "".join(x)))

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
#tfidf genre lists
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(songDF['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
genre_df.reset_index(drop = True, inplace=True)

In [ ]:
#simple function to create OHE features
#this gets passed later on
def ohe_prep(df, column, new_name): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        new_name (str): new column name to be used
        
    Returns: 
        tf_df: One hot encoded features 
    """
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [5]:
#function to build entire feature set
def create_feature_set(df, float_cols, other_cols):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    
    #tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    genre_df.reset_index(drop = True, inplace=True)

    #explicity_ohe = ohe_prep(df, 'explicit','exp')    
    year_ohe = ohe_prep(df, 'year','year') * 0.5
    popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.15

    # scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2
    other = df[other_cols].reset_index(drop = True)

    #concanenate all features
    final = pd.concat([floats_scaled, other], axis = 1)
    #add song id
    final['id']=df['id'].values
    
    return final

In [6]:
float_cols = songDF.dtypes[songDF.dtypes == 'float64'].index.values
other_cols = ['key', 'mode', 'duration_ms_x']
complete_feature_set = create_feature_set(songDF, float_cols=float_cols, other_cols=other_cols)#.mean(axis = 0)
complete_feature_set.to_csv("../data/complete_feature.csv", index = False)


In [7]:
playlistDF_test = playlistDF[playlistDF['name']=="thinking of you"]
playlistDF_test.to_csv("../data/test_playlist.csv", index = False)

In [8]:
def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):
    """ 
    Summarize a user's playlist into a single vector

    Parameters: 
        complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1).mean(axis =0)
    #complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['id','date_added']], on = 'id', how = 'inner')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1)
    
    playlist_feature_set = complete_feature_set_playlist.sort_values('tempo',ascending=True)

    # most_recent_date = playlist_feature_set.iloc[0,-1]
    
    # for ix, row in playlist_feature_set.iterrows():
    #     playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)
    playlist_feature_set['weight'] = playlist_feature_set['tempo'].apply(lambda x: weight_factor ** (-x))
    # print(playlist_feature_set['weight'])

    playlist_feature_set_weighted = playlist_feature_set.copy()
    #print(playlist_feature_set_weighted.iloc[:,:-2].columns)

    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-2].mul(playlist_feature_set_weighted.weight,0))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-2]
    # playlist_feature_set_weighted_final['id'] = playlist_feature_set['id']

    return playlist_feature_set_weighted_final.sum(axis = 0), complete_feature_set_nonplaylist

In [9]:
complete_feature_set_playlist_vector_EDM, complete_feature_set_nonplaylist_EDM = generate_playlist_feature(complete_feature_set, playlistDF_test, 1.09)

In [10]:
complete_feature_set_nonplaylist_EDM

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,mode,duration_ms_x,id
0,0.182996,0.1626,0.168547,0.025156,0.006245,1.401005e-03,0.00942,0.162325,0.114421,4,0,226863,0UaMYEvWZi0ZqiDOoHU3YI
1,0.156680,0.1676,0.178715,0.023701,0.005000,5.025126e-03,0.04840,0.185170,0.130453,5,0,198800,6I9VzXrHxO9rA9A5euc8Ak
2,0.134413,0.1516,0.170210,0.043659,0.000478,0.000000e+00,0.01196,0.140481,0.090525,2,0,235933,0WqIKmW4BTrj3eJFmnCKMv
3,0.180567,0.1428,0.171892,0.029314,0.040361,4.703518e-05,0.01042,0.163727,0.092087,4,0,267266,1AWQoqb9bSvzTjaLralEkT
4,0.172672,0.1212,0.176541,0.014823,0.011265,0.000000e+00,0.06260,0.131062,0.086421,0,1,227600,1lzr43nnXAijIGYnCT8M8H
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34218,0.091903,0.1210,0.169468,0.006528,0.051205,0.000000e+00,0.02180,0.036072,0.142146,4,1,263253,5mCttXluvE0gaIArboAoFA
34219,0.104049,0.1484,0.183141,0.005364,0.054217,3.417085e-07,0.01700,0.094188,0.092113,8,1,246453,7jP7iCJ1cXVXYHWIkYtP6d
34220,0.097773,0.1062,0.172421,0.006320,0.059639,0.000000e+00,0.02640,0.026052,0.113000,8,1,248573,5n2EPOuHyolttBawvILAZI
34221,0.130769,0.1190,0.171191,0.006798,0.010944,3.336683e-06,0.01652,0.104008,0.074581,7,1,256666,66xZqAh7nMNprBVc0A70lJ


In [11]:
complete_feature_set_playlist_vector_EDM

danceability        4.784092e+00
energy              2.915793e+00
loudness            6.781846e+00
speechiness         6.169308e-01
acousticness        5.382142e+00
instrumentalness    3.946775e-01
liveness            1.206294e+00
valence             2.636048e+00
tempo               4.411459e+00
key                 2.220866e+02
mode                3.568599e+01
duration_ms_x       8.987322e+06
dtype: float64

In [12]:
def generate_playlist_recos(df, features, nonplaylist_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    print(nonplaylist_features.drop('id', axis = 1).values)
    print("--------")
    print(features.values.reshape(1, -1))
    print(cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1)))
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    # non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_40

In [13]:
edm_top40 = generate_playlist_recos(songDF, complete_feature_set_playlist_vector_EDM, complete_feature_set_nonplaylist_EDM)
edm_top40

[[1.82995951e-01 1.62600000e-01 1.68546665e-01 ... 4.00000000e+00
  0.00000000e+00 2.26863000e+05]
 [1.56680162e-01 1.67600000e-01 1.78714591e-01 ... 5.00000000e+00
  0.00000000e+00 1.98800000e+05]
 [1.34412955e-01 1.51600000e-01 1.70209986e-01 ... 2.00000000e+00
  0.00000000e+00 2.35933000e+05]
 ...
 [9.77732794e-02 1.06200000e-01 1.72421375e-01 ... 8.00000000e+00
  1.00000000e+00 2.48573000e+05]
 [1.30769231e-01 1.19000000e-01 1.71191409e-01 ... 7.00000000e+00
  1.00000000e+00 2.56666000e+05]
 [1.19635628e-01 1.10600000e-01 1.71650257e-01 ... 5.00000000e+00
  1.00000000e+00 2.41400000e+05]]
--------
[[4.78409209e+00 2.91579312e+00 6.78184606e+00 6.16930775e-01
  5.38214160e+00 3.94677517e-01 1.20629370e+00 2.63604831e+00
  4.41145870e+00 2.22086571e+02 3.56859886e+01 8.98732165e+06]]
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


/Users/enjuichang/opt/anaconda3/envs/recsys/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,danceability,...,tempo,type,id,uri,track_href,analysis_url,duration_ms_y,time_signature,artists_song,sim
17996,52,Michael Wong,09oeZoDXXHxQKZnGlewEQu,spotify:artist:26SQFo2qNNGOxh2PUAsTeO,童話,spotify:album:62ChQcrrVUpXaFhSlPCeY0,244968,童話,Chill,0.549,...,136.083,audio_features,09oeZoDXXHxQKZnGlewEQu,spotify:track:09oeZoDXXHxQKZnGlewEQu,https://api.spotify.com/v1/tracks/09oeZoDXXHxQ...,https://api.spotify.com/v1/audio-analysis/09oe...,244968,4,Michael Wong童話,1.0
14121,56,Kenny Chesney,1t2hJFgJyUPD2crOafMUEk,spotify:artist:3grHWM9bx2E9vwJCdlRv9O,You and Tequila,spotify:album:0EJVUQGqNRxYBBCS0OLqY7,242840,Hemingway's Whiskey,Tennessee,0.691,...,135.839,audio_features,1t2hJFgJyUPD2crOafMUEk,spotify:track:1t2hJFgJyUPD2crOafMUEk,https://api.spotify.com/v1/tracks/1t2hJFgJyUPD...,https://api.spotify.com/v1/audio-analysis/1t2h...,242840,4,Kenny ChesneyYou and Tequila,1.0
47543,59,Mali Music,7gThA2feOdNSazklM0ze9Y,spotify:artist:4S4kD5NBlgaq4YLBQSEMyY,Johnny & Donna,spotify:album:7MMjwKpaSrATroOvGcjTGv,242160,Mali Is...,Smooth,0.462,...,86.053,audio_features,7gThA2feOdNSazklM0ze9Y,spotify:track:7gThA2feOdNSazklM0ze9Y,https://api.spotify.com/v1/tracks/7gThA2feOdNS...,https://api.spotify.com/v1/audio-analysis/7gTh...,242160,1,Mali MusicJohnny & Donna,1.0
40445,0,Tenth Avenue North,6jAAYLKxXPyeEoddXu6Gf7,spotify:artist:7v35CkHBaT1SStgQHl95a0,Worn,spotify:album:3TunW3H9Be9xvvR9Ip7a0L,244120,The Struggle,Worship,0.510,...,125.898,audio_features,6jAAYLKxXPyeEoddXu6Gf7,spotify:track:6jAAYLKxXPyeEoddXu6Gf7,https://api.spotify.com/v1/tracks/6jAAYLKxXPye...,https://api.spotify.com/v1/audio-analysis/6jAA...,244120,4,Tenth Avenue NorthWorn,1.0
2188,2,Jeremih,08zJpaUQVi9FrKv2e32Bah,spotify:artist:3KV3p5EY4AvKxOlhGHORLg,Planez,spotify:album:7DMyQuDPe8xzjC0UDSDa96,240320,Late Nights: The Album,abby,0.688,...,129.336,audio_features,08zJpaUQVi9FrKv2e32Bah,spotify:track:08zJpaUQVi9FrKv2e32Bah,https://api.spotify.com/v1/tracks/08zJpaUQVi9F...,https://api.spotify.com/v1/audio-analysis/08zJ...,240320,4,JeremihPlanez,1.0
27384,195,Faith Hill,4uYamklPK5p87CZgrQDFD3,spotify:artist:25NQNriVT2YbSW80ILRWJa,I Can't Do That Anymore,spotify:album:6Ki8tNYdetzeYqijkDlmds,242373,It Matters To Me,Rap/Pop,0.569,...,77.378,audio_features,4uYamklPK5p87CZgrQDFD3,spotify:track:4uYamklPK5p87CZgrQDFD3,https://api.spotify.com/v1/tracks/4uYamklPK5p8...,https://api.spotify.com/v1/audio-analysis/4uYa...,242373,4,Faith HillI Can't Do That Anymore,1.0
58618,41,Mariah Carey,42sGWmbhcUc1IeY7FxHb7s,spotify:artist:4iHNK0tOyZPYnBU7nGAgpQ,Joy Ride,spotify:album:43iBTEWECK7hSnE0p6GgNo,243800,The Emancipation of Mimi,Throwback,0.583,...,120.161,audio_features,42sGWmbhcUc1IeY7FxHb7s,spotify:track:42sGWmbhcUc1IeY7FxHb7s,https://api.spotify.com/v1/tracks/42sGWmbhcUc1...,https://api.spotify.com/v1/audio-analysis/42sG...,243800,4,Mariah CareyJoy Ride,1.0
57185,44,Heart,7lbazyJ26MIeptA0OW0sz0,spotify:artist:34jw2BbxjoYalTp8cJFCPv,No Other Love,spotify:album:7epNdy6eW0QnUzIwPLLCou,242200,Jupiters Darling,Mia,0.594,...,127.987,audio_features,7lbazyJ26MIeptA0OW0sz0,spotify:track:7lbazyJ26MIeptA0OW0sz0,https://api.spotify.com/v1/tracks/7lbazyJ26MIe...,https://api.spotify.com/v1/audio-analysis/7lba...,242200,3,HeartNo Other Love,1.0
44359,223,Taylor Swift,4NAmRvqSITAAzKWnC8yRq3,spotify:artist:06HL4z0CvFAxyc27GXpf02,Everything Has Changed,spotify:album:1EoDsNmgTLtmwe1BDAVxV5,243933,Red,Favorite Songs,0.624,...,80.017,audio_features,4NAmRvqSITAAzKWnC8yRq3,spotify:track:4NAmRvqSITAAzKWnC8yRq3,https://api.spotify.com/v1/tracks/4NAmRvqSITAA...,https://api.spotify.com/v1/audio-analysis/4NAm...,243933,4,Taylor SwiftEverything Has Changed,1.0
55151,55,David Foster,3FUJGbUvwM8IooDXaY8mN1,spotify:artist:0SgQK24WzZf2pXBXYqHJYF,The Best Of Me,spotify:album:61nRYw38onfZeQ2pjlrYp5,245760,David Foster,Wedding Dance Music,0.548,...,115.310,audio_features,3FUJGbUvwM8IooDXaY8mN1,spotify:track:3FUJGbUvwM8IooDXaY8mN1,https://api.spoti

In [217]:
print(edm_top40['sim'].unique())


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [221]:
x1 = [[ 1.82995951e-01, 1.62600000e-01, 1.68546665e-01,  2.51559252e-02,
   6.24497992e-03,  1.40100503e-03,  9.42000000e-03,  1.62324649e-01,
   1.14421082e-01,  8.13000000e-01,  4.00000000e+00, -7.10500000e+00,
   0.00000000e+00,  2.26863000e+05]]
   
y = [[ 4.78409209e+00, 2.91579312e+00, 6.78184606e+00, 6.16930775e-01,
  5.38214160e+00, 3.94677517e-01, 1.20629370e+00, 2.63604831e+00,
  4.41145870e+00, 2.22086571e+02, 3.56859886e+01, 8.98732165e+06]]

x2 = [[1.56680162e-01, 1.67600000e-01, 1.78714591e-01, 2.37006237e-02,
  5.00000000e-03, 5.02512563e-03, 4.84000000e-02, 1.85170341e-01,
  1.30453221e-01, 5.00000000e+00, 0.00000000e+00, 1.98800000e+05]]

cosine_similarity(x2,y)

array([[1.]])